In [216]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

Leemos los datos descargados con ayuda del API de idealista, al costado de cada csv se indica el mes de descarga de cada Dataframe.

In [217]:
path='C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/TFM/'
df_02=pd.read_csv(path+"idealista_02.csv")
df_03=pd.read_csv(path+"idealista_03.csv")
df_04=pd.read_csv(path+"idealista_04.csv")
df_05=pd.read_csv(path+"idealista_05.csv")


Agregamos una columna de mes a cada DataFrame y eliminamos la columna 'priceInfo'

In [218]:
df_02['Month']='Febrero'
df_03['Month']='Marzo'
df_04['Month']='Abril'
df_05['Month']='Mayo'

columnas=['index', 'propertyCode', 'thumbnail', 'externalReference', 'numPhotos',
       'floor', 'price', 'propertyType', 'operation', 'size', 'exterior',
       'rooms', 'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'distance', 'description', 'hasVideo', 'status',
       'newDevelopment', 'hasLift', 'priceByArea', 'detailedType',
       'suggestedTexts', 'hasPlan', 'has3DTour', 'has360', 'hasStaging',
       'topNewDevelopment', 'topPlus', 'parkingSpace',
       'newDevelopmentFinished', 'highlight', 'Month']

df_02=df_02[columnas]
df_03=df_03[columnas]
df_04=df_04[columnas]
df_05=df_05[columnas]

Verificamos que cada Dataframe tiene la misma estructura    

In [219]:
print(df_02.shape)
print(df_03.shape)
print(df_04.shape)
print(df_05.shape)

(5000, 42)
(4750, 42)
(4650, 42)
(5000, 42)


Unimos los Dataframes y eliminanos los duplicados en función de la columna 'propertyCode'manteniendo la última oferta de idealista 

In [220]:
# Unir los DataFrames en una lista
dfs = [df_02,df_03,df_04,df_05]

# Concatenar los DataFrames
merged_df = pd.concat(dfs,ignore_index=True)

# Eliminar filas duplicadas basadas en la columna 'propertyCode', manteniendo la última aparición
merged_df = merged_df.drop_duplicates(subset='propertyCode', keep='last')

merged_df.shape

(14993, 42)

Verificamos que no existan duplicados en merged_df

In [221]:
# Verificar si hay valores duplicados en la columna 'propertyCode'
duplicated_values = merged_df['propertyCode'].duplicated(keep=False)

# Mostrar las filas que tienen valores duplicados
duplicated_rows = merged_df[duplicated_values]

# Contar cuántas veces se repite cada valor duplicado
duplicate_counts = merged_df['propertyCode'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]

# Mostrar resultados
print("Filas con valores duplicados en 'propertyCode':")
print(duplicated_rows)

print("\nConteo de valores duplicados en 'propertyCode':")
print(duplicate_counts)

Filas con valores duplicados en 'propertyCode':
Empty DataFrame
Columns: [index, propertyCode, thumbnail, externalReference, numPhotos, floor, price, propertyType, operation, size, exterior, rooms, bathrooms, address, province, municipality, district, country, neighborhood, latitude, longitude, showAddress, url, distance, description, hasVideo, status, newDevelopment, hasLift, priceByArea, detailedType, suggestedTexts, hasPlan, has3DTour, has360, hasStaging, topNewDevelopment, topPlus, parkingSpace, newDevelopmentFinished, highlight, Month]
Index: []

[0 rows x 42 columns]

Conteo de valores duplicados en 'propertyCode':
Series([], Name: count, dtype: int64)


Hacemos un groub by para ver la distribución de las propiedades por municipio

In [222]:
merged_df.groupby('municipality').count()['province']

municipality
Getafe                    2
Leganés                  89
Madrid                14551
Pozuelo de Alarcón      351
Name: province, dtype: int64

Filtramos el Dataframe para tener solo las propiedades dentro de la municipalidad de Madrid, y las variables a trabajar. Finalmente eliminamos cualquier oferta que tenga un NaN.

In [1]:
filtered_df=merged_df[merged_df['municipality']=='Madrid']

variables=['propertyCode','price', 'floor', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms','district','neighborhood','latitude', 'longitude', 'distance',
           'status', 'newDevelopment', 'hasLift','priceByArea', 'parkingSpace', 'newDevelopmentFinished']

filtered_df=filtered_df[variables]

filtered_df


NameError: name 'merged_df' is not defined

In [224]:
# Vemos los tipos de datos para cada columna si es númerico, bool u object
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14551 entries, 0 to 19399
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   propertyCode            14551 non-null  int64  
 1   price                   14551 non-null  float64
 2   floor                   13339 non-null  object 
 3   propertyType            14551 non-null  object 
 4   size                    14551 non-null  float64
 5   exterior                13279 non-null  object 
 6   rooms                   14551 non-null  int64  
 7   bathrooms               14551 non-null  int64  
 8   district                14551 non-null  object 
 9   neighborhood            14551 non-null  object 
 10  latitude                14551 non-null  float64
 11  longitude               14551 non-null  float64
 12  distance                14551 non-null  int64  
 13  status                  14551 non-null  object 
 14  newDevelopment          14551 non-null  boo

Corregimos los datos de la columna floor

In [225]:
# Reemplazar 'bj' y 'en' por '0' en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace(['bj','en'], 0)

# Reemplazar 'ss' por '-1' en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace('ss', '-1')

# Eliminamos las 'st' ya que son sobretecho y no sabes cual es el valor de la última planta de cada piso
filtered_df = filtered_df.drop(filtered_df[filtered_df['floor'] == 'st'].index)

# Cambiar los valores de 60 a 6 en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace('60', '6')

# Convertir la columna 'floor' de tipo 'object' a tipo 'integer', manteniendo NaNs
filtered_df['floor'] = pd.to_numeric(filtered_df['floor'], errors='coerce')

# Calcular la mediana de 'floor' para cada 'propertyType'
median_floors = filtered_df.groupby('propertyType')['floor'].median()

# Definir una función para rellenar NaNs con la mediana correspondiente
def fill_floor(row):
    if pd.isna(row['floor']):
        if row['propertyType'] == 'chalet':
            return 0
        else:
            return median_floors[row['propertyType']]
    else:
        return row['floor']

# Aplicar la función para rellenar NaNs
filtered_df['floor'] = filtered_df.apply(fill_floor, axis=1)

# Convertir la columna 'floor' a tipo 'int'
filtered_df['floor'] = filtered_df['floor'].astype(int)

filtered_df['floor'].unique()


array([ 2,  0,  3,  1,  4, -1,  5,  7,  8, 13,  6, 10,  9, 18, 12, 22, 11,
       15, 14, 19, 17, 20, 24, 16, 21, -2])

Corregimos los datos de la columna exterior

In [226]:
# Reemplazar 'Nan' por 'False' en la columna 'exterior'
filtered_df.fillna({'exterior': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'exterior'
filtered_df['exterior'] = filtered_df['exterior'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'exterior'
filtered_df['exterior'] = filtered_df['exterior'].replace('False', 0)

# Convertir la columna 'exterior' de tipo 'object' a tipo 'integer'
filtered_df['exterior'] = filtered_df['exterior'].astype(int)

# Mostrar el resultado
filtered_df['exterior'].unique()

array([1, 0])

Corregimos los datos de la columna rooms

In [227]:
# Incrementar en una la cantidad de habitaciones en los studios que tienen 0 habitaciones.
filtered_df['rooms'] = filtered_df.apply(lambda row: row['rooms'] + 1 if row['propertyType'] == 'studio' else row['rooms'], axis=1)

# Eliminar filas donde 'rooms' o 'bathrooms' es 0
filtered_df = filtered_df.drop(filtered_df[(filtered_df['rooms'] == 0) | (filtered_df['bathrooms'] == 0)].index)

filtered_df['rooms'].unique()

array([ 4,  3,  2,  1,  5,  7, 12,  6,  8, 11,  9, 15, 10, 13],
      dtype=int64)

Corregimos los datos de la columna newDevelopment

In [228]:
# Reemplazar 'True' por '1' en la columna 'newDevelopment'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'newDevelopment'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].replace('False', 0)

# Convertir la columna 'newDevelopment' de tipo 'object' a tipo 'integer'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].astype(int)

# Mostrar el resultado
filtered_df['newDevelopment'].unique()

array([0, 1])

Corregimos los datos de la columna hasLift

In [229]:
# Reemplazar 'Nan' por 'False' en la columna 'hasLift'
filtered_df.fillna({'hasLift': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'hasLift'
filtered_df['hasLift'] = filtered_df['hasLift'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'hasLift'
filtered_df['hasLift'] = filtered_df['hasLift'].replace('False', 0)

# Convertir la columna 'hasLift' de tipo 'object' a tipo 'integer'
filtered_df['hasLift'] = filtered_df['hasLift'].astype(int)

# Mostrar el resultado
filtered_df['hasLift'].unique()

array([1, 0])

Corregimos los datos de la columna newDevelopmentFinished


In [230]:
# Reemplazar 'Nan' por 'False' en la columna 'newDevelopmentFinished'
filtered_df.fillna({'newDevelopmentFinished': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'newDevelopmentFinished'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'newDevelopmentFinished'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].replace('False', 0)

# Convertir la columna 'newDevelopmentFinished' de tipo 'object' a tipo 'integer'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].astype(int)

# Mostrar el resultado
filtered_df['newDevelopmentFinished'].unique()

array([0, 1])

Corregimos los datos de la columna parkingSpace
y generamos tres columnas : hasParkingSpace , isParkingSpaceIncludedInPrice , parkingSpacePrice

In [231]:
import ast

# Función para procesar cada fila
def process_row(row):
    if pd.isna(row):
        return pd.Series({'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 0})
    else:
        try:
            row_dict = ast.literal_eval(row)  # Convertir la cadena a un diccionario
            has_parking_space = row_dict.get('hasParkingSpace', False)
            is_included = row_dict.get('isParkingSpaceIncludedInPrice', False)
            price = row_dict.get('parkingSpacePrice', 0)
        except (SyntaxError, ValueError):  # Manejar el caso de que la cadena no sea un diccionario válido
            has_parking_space = False
            is_included = False
            price = 0
        return pd.Series({'hasParkingSpace': has_parking_space, 'isParkingSpaceIncludedInPrice': is_included, 'parkingSpacePrice': price})

# Aplicar la función a cada fila
new_columns = filtered_df['parkingSpace'].apply(process_row)

# Concatenar los resultados al DataFrame original
filtered_df = pd.concat([filtered_df, new_columns], axis=1)

# Eliminar la columna 'parkingSpace' tras generar las tres columnas
filtered_df = filtered_df.drop(columns=['parkingSpace'])

Corregimos los datos de la columna hasParkingSpace

In [232]:
# Reemplazar 'Nan' por 'False' en la columna 'hasParkingSpace'
filtered_df.fillna({'hasParkingSpace': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'hasParkingSpace'
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'hasParkingSpace'
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].replace('False', 0)

# Convertir la columna 'hasParkingSpace' de tipo 'object' a tipo 'integer' 
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].astype(int)

# Mostrar el resultado
filtered_df['hasParkingSpace'].unique()

array([0, 1])

Corregimos los datos de la columna isParkingSpaceIncludedInPrice

In [233]:
# Reemplazar 'Nan' por 'False' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df.fillna({'isParkingSpaceIncludedInPrice': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].replace('False', 0)

# Convertir la columna 'isParkingSpaceIncludedInPrice' de tipo 'object' a tipo 'integer' 
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].astype(int)

# Mostrar el resultado
filtered_df['isParkingSpaceIncludedInPrice'].unique()

array([0, 1])

Media del precio m2 por distrito

In [234]:
# Calcular la media de priceByArea por distrito
mean_price_by_district = filtered_df[['district', 'priceByArea']].groupby('district').mean()

# Crear un diccionario a partir de la serie resultante
mean_price_dict = mean_price_by_district['priceByArea'].to_dict()

# Crear la nueva columna priceByAreaByDistrict en el DataFrame original
filtered_df['priceByAreaByDistrict'] = filtered_df['district'].map(mean_price_dict)

# Mostrar los primeros resultados para verificar
filtered_df

,propertyCode,price,floor,propertyType,size,exterior,rooms,bathrooms,district,neighborhood,...,distance,status,newDevelopment,hasLift,priceByArea,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,priceByAreaByDistrict
0,95642317,1197000.0,2,flat,217.0,1,4,3,Centro,Sol,...,375,renew,0,1,5516.0,0,0,0,0.0,6848.457760
1,103827609,91000.0,0,flat,85.0,1,3,2,Villaverde,San Cristóbal,...,8212,renew,0,0,1071.0,0,0,0,0.0,2087.688940
2,103187538,279000.0,3,flat,129.0,1,2,2,Fuencarral,Pilar,...,6373,good,0,1,2163.0,0,0,0,0.0,4200.077758
4,103789477,350000.0,1,flat,93.0,1,3,1,Arganzuela,Palos de Moguer,...,1814,good,0,1,3763.0,0,0,0,0.0,4742.169896
5,103845902,70700.0,4,flat,113.0,1,3,2,Puente de Vallecas,Palomeras Bajas,...,5657,good,0,1,626.0,0,0,0,0.0,2412.210210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19395,104316258,699000.0,1,flat,145.0,1,3,2,Hortaleza,Sanchinarro,...,9891,good,0,1,4821.0,0,1,1,0.0,4454.571885
19396,103842038,299000.0,0,flat,105.0,1,3,1,Tetuán,Ventilla-Almenara,...,5574,good,0,1,2848.0,0,0,0,0.0,5065.689011
19397,104715802,359900.0,2,flat,70.0,1,2,2,Tetuán,Valdeacederas,...,5636,newdevelopment,1,1,5141.0,0,1,1,0.0,5065.689011
19398,104811783,306000.0,2,flat,95.0,1,2,2,Vicálvaro,Ambroz,...,8448,newdevelopment,1,1,3221.0,0,0,0,0.0,2669.257812


Media del precio m2 por barrio

In [235]:
# Calcular la media de priceByArea por distrito
mean_price_by_neighborhood = filtered_df[['neighborhood', 'priceByArea']].groupby('neighborhood').mean()

# Crear un diccionario a partir de la serie resultante
mean_price_dict = mean_price_by_neighborhood['priceByArea'].to_dict()

# Crear la nueva columna priceByAreaByneighborhood en el DataFrame original
filtered_df['priceByAreaByneighborhood'] = filtered_df['neighborhood'].map(mean_price_dict)

# Mostrar los primeros resultados para verificar
filtered_df

,propertyCode,price,floor,propertyType,size,exterior,rooms,bathrooms,district,neighborhood,...,status,newDevelopment,hasLift,priceByArea,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice,priceByAreaByDistrict,priceByAreaByneighborhood
0,95642317,1197000.0,2,flat,217.0,1,4,3,Centro,Sol,...,renew,0,1,5516.0,0,0,0,0.0,6848.457760,7704.791962
1,103827609,91000.0,0,flat,85.0,1,3,2,Villaverde,San Cristóbal,...,renew,0,0,1071.0,0,0,0,0.0,2087.688940,1290.988764
2,103187538,279000.0,3,flat,129.0,1,2,2,Fuencarral,Pilar,...,good,0,1,2163.0,0,0,0,0.0,4200.077758,3797.011628
4,103789477,350000.0,1,flat,93.0,1,3,1,Arganzuela,Palos de Moguer,...,good,0,1,3763.0,0,0,0,0.0,4742.169896,4924.068493
5,103845902,70700.0,4,flat,113.0,1,3,2,Puente de Vallecas,Palomeras Bajas,...,good,0,1,626.0,0,0,0,0.0,2412.210210,2239.417582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19395,104316258,699000.0,1,flat,145.0,1,3,2,Hortaleza,Sanchinarro,...,good,0,1,4821.0,0,1,1,0.0,4454.571885,4973.384615
19396,103842038,299000.0,0,flat,105.0,1,3,1,Tetuán,Ventilla-Almenara,...,good,0,1,2848.0,0,0,0,0.0,5065.689011,4159.815789
19397,104715802,359900.0,2,flat,70.0,1,2,2,Tetuán,Valdeacederas,...,newdevelopment,1,1,5141.0,0,1,1,0.0,5065.689011,4628.348718
19398,104811783,306000.0,2,flat,95.0,1,2,2,Vicálvaro,Ambroz,...,newdevelopment,1,1,3221.0,0,0,0,0.0,2669.257812,2371.714286


Verificamos que los datos númericos y categoricos estén correctamente.

In [236]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14466 entries, 0 to 19399
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   propertyCode                   14466 non-null  int64  
 1   price                          14466 non-null  float64
 2   floor                          14466 non-null  int32  
 3   propertyType                   14466 non-null  object 
 4   size                           14466 non-null  float64
 5   exterior                       14466 non-null  int32  
 6   rooms                          14466 non-null  int64  
 7   bathrooms                      14466 non-null  int64  
 8   district                       14466 non-null  object 
 9   neighborhood                   14466 non-null  object 
 10  latitude                       14466 non-null  float64
 11  longitude                      14466 non-null  float64
 12  distance                       14466 non-null  int6

Guardamos el Dataframe procesado en un archivo csv

In [237]:
#Definimos el nombre del archivo
nombre_cvs='datos_procesados_11.csv'

# Definimos la ruta
ruta='C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/TFM/'

# Ruta donde guardar el archivo CSV
ruta_csv = ruta+nombre_cvs

# Guardar el DataFrame en un archivo CSV
filtered_df.to_csv(ruta_csv, index=False)